CRAWL DATA

In [67]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd

In [68]:
# Set up Selenium WebDriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # Run in headless mode
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

In [69]:
driver.get("https://www.worldometers.info/coronavirus/#countries")
time.sleep(3)  # Wait for the page to load

In [70]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

try:
    # Chờ phần tử xuất hiện
    two_days_ago_button = WebDriverWait(driver, 15).until(
        EC.element_to_be_clickable((By.XPATH, "//a[@href='#nav-yesterday2']"))
    )

    # Click vào nút
    driver.execute_script("arguments[0].click();", two_days_ago_button)
    print("Clicked on '2 Days Ago' successfully!")

except Exception as e:
    print("Error clicking '2 Days Ago':", e)

Clicked on '2 Days Ago' successfully!


In [71]:
try:
    columns_button = driver.find_element(By.XPATH, "//button[contains(text(), 'Columns')]")
    driver.execute_script("arguments[0].click();", columns_button)
    time.sleep(5)
except Exception as e:
    print("Error clicking 'Columns':", e)

In [72]:
# Click on "New Cases" column header to sort by new cases
try:
    new_cases_button = driver.find_element(By.XPATH, "//label[@for='column_3']")
    driver.execute_script("arguments[0].click();", new_cases_button)
    time.sleep(5)
except Exception as e:
    print("Error clicking 'New Cases':", e)

In [73]:
elements = driver.find_elements(By.XPATH, "//table[@id='main_table_countries_yesterday2']/tbody/tr")

In [74]:
print(f"Number of rows found: {len(elements)}")
data = []
for row in elements:
    cols = row.find_elements(By.TAG_NAME, "td") 
    if cols:
        print([col.text.strip() for col in cols])  # In dữ liệu thô
        data.append([col.text.strip() for col in cols])

Number of rows found: 247
['', 'World', '704,749,931', '+3,536', '7,010,624', '675,604,068', '90,413', '899.4', '', '']
['', '', '', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', '']
['1', 'USA', '111,818,931', '+1,744', '1,219,480', '109,804,319', '333,982', '3,642', '334,805,269', '']
['', '', '', '', '', '', '', '', '', '']
['2', 'India', '45,035,325', '+71', '533,570', 'N/A', '32,016', '379', '1,406,631,776', '']
['3', 'France', '40,138,560', '', '167,642', '39,970,918', '612,013', '2,556', '65,584,518', '']
['4', 'Germany', '38,828,922', '+746', '183,011', '38,240,600', '462,891', '2,182', '83,883,596', '']
['5', 'Brazil', '38,743,918', '', '711,380', '36,249,161', '179,908', '3,303', '215,353,593', '']
['6', 'S. Korea', '34,571,873', '', '35,934', '34,535,939', '673,523', '700', '51,329,899', '']
['7', 'Japan', '33,803,572', '', '74,694', 'N/A', '269,169', '595', '125,584,838', '']
['8', 'Italy', '26,723,249', '', '196,4

In [75]:
# Close WebDriver
driver.quit()

In [76]:
#Xử lý dữ liệu đã crawl
data = [row[:9] for row in data]
data = [row for row in data if any(cell.strip() for cell in row)]

In [77]:
# Create DataFrame
columns = ["#", "Country Other", "Total Cases", "New Cases", "Total Deaths", "Total Recovered", "Cases/1M Pop", "Deaths/1M Pop", "Population"]
df = pd.DataFrame(data, columns=columns)

In [78]:
df

,#,Country Other,Total Cases,New Cases,Total Deaths,Total Recovered,Cases/1M Pop,Deaths/1M Pop,Population
0,,World,"704,749,931","+3,536","7,010,624","675,604,068","90,413",899.4,
1,1,USA,"111,818,931","+1,744","1,219,480","109,804,319","333,982","3,642","334,805,269"
2,2,India,"45,035,325",+71,"533,570",N/A,"32,016",379,"1,406,631,776"
3,3,France,"40,138,560",,"167,642","39,970,918","612,013","2,556","65,584,518"
4,4,Germany,"38,828,922",+746,"183,011","38,240,600","462,891","2,182","83,883,596"
...,...,...,...,...,...,...,...,...,...
228,228,Tokelau,80,,,,"58,055",,"1,378"
229,229,Vatican City,29,,,29,"36,295",,799
230,230,Western Sahara,10,,1,9,16,2,"626,161"
231,231,MS Zaandam,9,,2,7,,,


In [79]:
# Convert numeric columns to proper format
def convert_numeric(value):
    return int(value.replace(",", "")) if value.replace(",", "").isdigit() else value
df = df.applymap(convert_numeric)

# Display the DataFrame
print(df)

       #   Country Other  Total Cases New Cases Total Deaths Total Recovered  \
0                  World    704749931    +3,536      7010624       675604068   
1      1             USA    111818931    +1,744      1219480       109804319   
2      2           India     45035325       +71       533570             N/A   
3      3          France     40138560                 167642        39970918   
4      4         Germany     38828922      +746       183011        38240600   
..   ...             ...          ...       ...          ...             ...   
228  228         Tokelau           80                                          
229  229    Vatican City           29                                     29   
230  230  Western Sahara           10                      1               9   
231  231      MS Zaandam            9                      2               7   
232               Total:    704749931    +3,536      7010624                   

    Cases/1M Pop Deaths/1M Pop  Populat

C:\Users\Thy\AppData\Local\Temp\ipykernel_9908\542521856.py:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(convert_numeric)


In [80]:
# Save to CSV
df.to_csv("covid_data.csv", index=False)

BAI TAP 1

In [81]:
# Câu 1
data_copy = df.copy()
data_copy = data_copy.drop(index = 0)
data_copy = data_copy.drop(index = 232)
data_copy

,#,Country Other,Total Cases,New Cases,Total Deaths,Total Recovered,Cases/1M Pop,Deaths/1M Pop,Population
1,1,USA,111818931,"+1,744",1219480,109804319,333982,3642,334805269
2,2,India,45035325,+71,533570,N/A,32016,379,1406631776
3,3,France,40138560,,167642,39970918,612013,2556,65584518
4,4,Germany,38828922,+746,183011,38240600,462891,2182,83883596
5,5,Brazil,38743918,,711380,36249161,179908,3303,215353593
...,...,...,...,...,...,...,...,...,...
227,227,Diamond Princess,712,,13,699,,,
228,228,Tokelau,80,,,,58055,,1378
229,229,Vatican City,29,,,29,36295,,799
230,230,Western Sahara,10,,1,9,16,2,626161


In [82]:
Top_5_countries = data_copy.sort_values(by = 'Total Cases', ascending = False).head()
Top_5_countries

,#,Country Other,Total Cases,New Cases,Total Deaths,Total Recovered,Cases/1M Pop,Deaths/1M Pop,Population
1,1,USA,111818931,"+1,744",1219480,109804319,333982,3642,334805269
2,2,India,45035325,+71,533570,N/A,32016,379,1406631776
3,3,France,40138560,,167642,39970918,612013,2556,65584518
4,4,Germany,38828922,+746,183011,38240600,462891,2182,83883596
5,5,Brazil,38743918,,711380,36249161,179908,3303,215353593


In [83]:
# Câu 2
Top_1_countries = data_copy.sort_values(by = 'New Cases', ascending = False).head()
Top_1_countries.iloc[0]

#                         13
Country Other      Australia
Total Cases         11852480
New Cases               +954
Total Deaths           24414
Total Recovered     11815715
Cases/1M Pop          454662
Deaths/1M Pop            937
Population          26068792
Name: 13, dtype: object

In [85]:
# Chuyển đổi các giá trị N/A hoặc không hợp lệ thành NaN, sau đó ép kiểu thành int
data_copy['Total Recovered'] = pd.to_numeric(data_copy['Total Recovered'], errors='coerce')

# Thay thế NaN bằng giá trị mặc định (nếu cần, ví dụ: 0)
data_copy['Total Recovered'] = data_copy['Total Recovered'].fillna(0).astype(int)

# Kiểm tra kết quả
data_copy

,#,Country Other,Total Cases,New Cases,Total Deaths,Total Recovered,Cases/1M Pop,Deaths/1M Pop,Population
1,1,USA,111818931,"+1,744",1219480,109804319,333982,3642,334805269
2,2,India,45035325,+71,533570,0,32016,379,1406631776
3,3,France,40138560,,167642,39970918,612013,2556,65584518
4,4,Germany,38828922,+746,183011,38240600,462891,2182,83883596
5,5,Brazil,38743918,,711380,36249161,179908,3303,215353593
...,...,...,...,...,...,...,...,...,...
227,227,Diamond Princess,712,,13,699,,,
228,228,Tokelau,80,,,0,58055,,1378
229,229,Vatican City,29,,,29,36295,,799
230,230,Western Sahara,10,,1,9,16,2,626161


In [86]:
# Tính tỉ lệ tổng số ca bình phục trên tổng số ca nhiễm
data_copy['Recovery Rate'] = data_copy['Total Recovered'] / data_copy['Total Cases']

# Sắp xếp theo tỉ lệ bình phục giảm dần và lấy 3 quốc gia có tỉ lệ cao nhất
Top_3_recovery_countries = data_copy.sort_values(by='Recovery Rate', ascending=False).head(3)

# Hiển thị kết quả
Top_3_recovery_countries

,#,Country Other,Total Cases,New Cases,Total Deaths,Total Recovered,Cases/1M Pop,Deaths/1M Pop,Population,Recovery Rate
224,224,Falkland Islands,1930,,,1930,545352,,3539,1.000000
229,229,Vatican City,29,,,29,36295,,799,1.000000
32,32,DPRK,4772813,,74,4772739,183636,3,25990679,0.999984
